In [18]:
from openbabel import openbabel

In [19]:
from rdkit import Chem
from rdkit.Chem import rdBase
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Descriptors
from rdkit.Chem import Crippen

In [20]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as ss

In [56]:
from ast import literal_eval

In [21]:
from treelib import Node, Tree

In [22]:
class Molecule(object):
    def __init__(self, path):
        self.path = path

In [23]:
def split(word):
    return[char for char in word]

In [83]:
def index_finder(Product, rels, path):
    index = 0
    for i in range(len(rels['Index'])):
        found_index = False
        place = literal_eval(rels['Products'][i])
        for j in range(len(place)):
            if place[j] == Product:
                valid = True
                precursors = precursor_finder(rels['Index'][i], rels)
                for k in range(len(precursors)):
                    if precursors[k] in path:
                        valid = False
                        break
                if valid == True:
                    found_index = True
            if found_index == True:
                index = rels['Index'][i]
                print(rels['Index'][i])
                print(precursors)
                return(index)
                break

In [84]:
def precursor_finder(index, rels):
    precursors = []
    for i in range(len(rels['Index'])):
        if rels['Index'][i] == index:
            dummy = literal_eval(rels['Reagents'][i])
            for j in range(len(dummy)):
                precursors.append(dummy[j])
    return(precursors)

In [85]:
def find_component(node, rels):
    Product = node.tag
    path = (node.data).path
    index = index_finder(Product, rels, path)
    precursors = precursor_finder(index, rels)
    return(precursors)

In [86]:
def map_tree(Smiles, rels):
    rels = pd.read_csv(rels, sep='\t')
    base_molecules = ['N', 'C=O', 'C(CO)=O', 'O'] #FormoseAmm
    #base_molecules = ['C(C(C)=O)(O)=O', 'O'] #PyruvicAcid
    tree = Tree()
    tree.create_node(Smiles, 0, data=Molecule([Smiles]))
    finished = False
    level_counter = 0
    node_counter = 0
    while finished == False:
        print(level_counter)
        nodes = tree.all_nodes()
        active_nodes = []
        for i in range(len(nodes)):
            tag = nodes[i].tag
            level = tree.depth(nodes[i])
            if level == level_counter and tag not in base_molecules:
                active_nodes.append(nodes[i])
        if active_nodes == []:
            finished = True
            break
        else:
            for i in range(len(active_nodes)):
                precursors = find_component(active_nodes[i], rels)
                product = active_nodes[i].identifier
                dummy = (active_nodes[i].data).path
                place = []
                for k in range(len(dummy)):
                    place.append(dummy[k])
                for j in range(len(precursors)):
                    node_counter +=1
                    tree.create_node(precursors[j], node_counter, parent=product, data=Molecule(place + [precursors[j]]))
            level_counter+=1
    return(tree)

In [87]:
%%time
a = map_tree('C(C(C(C)NCCC(C=O)=O)O)=O', './ProcessedRels/FormoseAmmProcessedRels.tsv')

0
43902_0
['C=CC(C=O)=O', 'C(C(C(C)N)O)=O']
1
1867_0
['C(C(CCO)=O)=O']
2193_0
['C(CN)(O)=O', 'C(C(C(C)=O)O)=O']
2
90_0
['C(C(C(CO)O)O)=O']
63_0
['C=O', 'O', 'C(CN)=O']
96_0
['C(C(C(CO)O)O)=O']
3
7_0
['C(CO)=O', 'C(CO)=O']
10_0
['N', 'C(CO)=O']
7_0
['C(CO)=O', 'C(CO)=O']
4
CPU times: user 10.6 s, sys: 67.3 ms, total: 10.7 s
Wall time: 10.7 s


In [88]:
a.show()

C(C(C(C)NCCC(C=O)=O)O)=O
├── C(C(C(C)N)O)=O
│   ├── C(C(C(C)=O)O)=O
│   │   └── C(C(C(CO)O)O)=O
│   │       ├── C(CO)=O
│   │       └── C(CO)=O
│   └── C(CN)(O)=O
│       ├── C(CN)=O
│       │   ├── C(CO)=O
│       │   └── N
│       ├── C=O
│       └── O
└── C=CC(C=O)=O
    └── C(C(CCO)=O)=O
        └── C(C(C(CO)O)O)=O
            ├── C(CO)=O
            └── C(CO)=O



In [81]:
a.show(data_property='path')

['C(C(CO)O)=O']
├── ['C(C(CO)O)=O', 'C(CO)=O']
└── ['C(C(CO)O)=O', 'C=O']

